In [1]:
import os
import glob
import numpy as np
from pint.models import get_model
import pint.logging
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import constants as const

pint.logging.setup(level="WARNING")

1

In [2]:
def deg_GBT(sep,cfreq,diam=100.0 * u.m):
    """ Calculate degradation given angular separation, center frequency
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. The default diameter is for the GBT and the beam's full width at
    half max is calculated using FWHM = 1.2*lamda/eff_diam rad.
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    cfreq: quantity, center frequency of observation (MHz)
    diam: optional quantity, effective diameter of telescope (meters)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    lamda = const.c/cfreq
    fwhm = 1.2*(lamda/diam*u.rad).to(u.arcmin)
    sig = fwhm/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [3]:
MHz = 1.0e6*u.Hz
pars = np.sort(glob.glob("*[0-9]_tdb.par")) # veto J1816 eclipse par and J0032 DMX par

In [12]:
data_dir = "./"

for p in pars:
    par_path = os.path.join(data_dir, p)
    mo = get_model(par_path)
    mlfrd_path = os.path.join(data_dir,f"{mo.PSR.value}.mlfrd")
    
    # mfrd files contain MJD, length, center frequency, RA, Dec info for each scan for a given pulsar
    with open(mlfrd_path) as f:
        mlfrd_lines = f.readlines()
    
    # For each scan, get the RA/Dec and compare to timing position (eqcoord)
    eqcoord = mo.coords_as_ICRS()
    
    fnames = []
    freqs = []
    obstime = 0
    separations = [] # for now, gather angular separations (arcmins)
    for ml in mlfrd_lines:
        if ml.startswith(('GUPPI', 'guppi', 'VEGAS', 'puppi', 'J1327')):
            fname, mjd_str, len_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freqs.append(float(freq_str))
            separations.append(eqcoord.separation(c).arcminute)
            obstime += float(len_str)
    
    # Closer look at separations
    separations = np.array(separations)
    degradations = np.array([deg_GBT(s*u.arcmin,f*MHz) for s,f in zip(separations,freqs)])
    n2000,n1500,n1430,n1380,n820,n430,n350,n327,n72,n57,n42 = freqs.count(2000.0),freqs.count(1500.0),freqs.count(1430.0),freqs.count(1380.0),freqs.count(820.0),freqs.count(430.0),freqs.count(350.0),freqs.count(327.0),freqs.count(71.85),freqs.count(57.15),freqs.count(42.45)
    nother = len(freqs) - (n42+n57+n72+n327+n350+n430+n820+n1380+n1430+n1500+n2000)
    print(f"PSR {mo.PSR.value} was observed {len(separations)} times")
    
    PRINT_FILES = True
    for ff in [42.45,57.15,71.85,327.0,350.0,430.0,820.0,1380.0,1430.0,1500.0,2000.0]:
        i_freq = np.where(np.array(freqs) == ff)[0]
        # separation thresholds (st) chosen to limit degradation to 10%
        if ff == 42.45:
            st = 56.8
            nt = n42
        if ff == 57.15:
            st = 42.2
            nt = n57
        if ff == 71.85:
            st = 33.5
            nt = n72
        if ff == 327.0:
            st = 7.5
            nt = n327
        if ff == 350.0:
            st = 7.0
            nt = n350
        if ff == 430.0:
            st = 5.7
            nt = n430
        if ff == 820.0:
            st = 3.0
            nt = n820
        if ff == 1380.0:
            st = 1.7
            nt = n1380
        if ff == 1430.0:
            st = 1.7
            nt = n1430
        if ff == 1500.0:
            st = 1.7
            nt = n1500
        if ff == 2000.0:
            st = 1.3
            nt = n2000
        nws = len(np.where(separations[i_freq]<st)[0]) # number within separation threshold
        if separations[i_freq].any():
            print(
            f"  {nws} time(s) at {ff:.0f} MHz within {st:.1f} arcmin of the timing position ({nt} total)"
            )
            minsep = np.min(separations[i_freq])
            mindeg = deg_GBT(minsep*u.arcmin,ff*MHz)
            medsep = np.median(separations[i_freq])
            meddeg = deg_GBT(medsep*u.arcmin,ff*MHz)
            print(f"  Min separation at {ff:.0f} MHz: {minsep:.1f} arcmin (deg = {mindeg:0.2f})")
            print(f"  Med separation at {ff:.0f} MHz: {medsep:.1f} arcmin (deg = {meddeg:0.2f})")
        else:
            if ff == 350.0 or ff == 820.0:
                print(f"  (No detections at {ff:.0f} MHz)")
            
        if PRINT_FILES:
            psr = p.split("_")[0]    
            sum_fname = f"{psr}_{int(ff)}.files"
            freq_cut = [(f==ff) for f in freqs]
            seps_cut = (separations < st)
            #print(np.array(fnames)[freq_cut*seps_cut])
            if nws != nt:
                with open(sum_fname,'w') as f:
                    for file in np.array(fnames)[freq_cut*seps_cut]:
                        f.write(file+'\n')
    print(f"  PSR {mo.PSR.value} has {nother} observations at unaccounted-for frequencies")

PSR J0032+6946 was observed 81 times
  13 time(s) at 350 MHz within 7.0 arcmin of the timing position (42 total)
  Min separation at 350 MHz: 3.3 arcmin (deg = 0.98)
  Med separation at 350 MHz: 8.4 arcmin (deg = 0.85)
  13 time(s) at 820 MHz within 3.0 arcmin of the timing position (39 total)
  Min separation at 820 MHz: 0.0 arcmin (deg = 1.00)
  Med separation at 820 MHz: 3.8 arcmin (deg = 0.84)
  PSR J0032+6946 has 0 observations at unaccounted-for frequencies
PSR J0141+6303 was observed 37 times
  16 time(s) at 350 MHz within 7.0 arcmin of the timing position (17 total)
  Min separation at 350 MHz: 0.0 arcmin (deg = 1.00)
  Med separation at 350 MHz: 6.5 arcmin (deg = 0.91)
  0 time(s) at 820 MHz within 3.0 arcmin of the timing position (20 total)
  Min separation at 820 MHz: 6.5 arcmin (deg = 0.60)
  Med separation at 820 MHz: 6.7 arcmin (deg = 0.58)
  PSR J0141+6303 has 0 observations at unaccounted-for frequencies
PSR J0214+5222 was observed 46 times
  15 time(s) at 350 MHz with

PSR J1930+6205 was observed 20 times
  0 time(s) at 350 MHz within 7.0 arcmin of the timing position (2 total)
  Min separation at 350 MHz: 13.1 arcmin (deg = 0.68)
  Med separation at 350 MHz: 13.7 arcmin (deg = 0.66)
  0 time(s) at 820 MHz within 3.0 arcmin of the timing position (18 total)
  Min separation at 820 MHz: 4.6 arcmin (deg = 0.77)
  Med separation at 820 MHz: 4.6 arcmin (deg = 0.77)
  PSR J1930+6205 has 0 observations at unaccounted-for frequencies
PSR J2104+2830 was observed 21 times
  0 time(s) at 350 MHz within 7.0 arcmin of the timing position (3 total)
  Min separation at 350 MHz: 18.6 arcmin (deg = 0.46)
  Med separation at 350 MHz: 20.9 arcmin (deg = 0.38)
  0 time(s) at 820 MHz within 3.0 arcmin of the timing position (18 total)
  Min separation at 820 MHz: 4.5 arcmin (deg = 0.78)
  Med separation at 820 MHz: 4.9 arcmin (deg = 0.74)
  PSR J2104+2830 has 0 observations at unaccounted-for frequencies
PSR J2115+6702 was observed 17 times
  0 time(s) at 350 MHz within